### 카메라에서 실시간으로 영상을 캡처

In [2]:
import cv2  

# 웹캠 연결 및 비디오 캡처 객체 생성 (0은 기본 카메라를 나타냄)
cam = cv2.VideoCapture(0)

# 웹캠이 열려있는 동안 계속 반복
while True:
    # 첫 번째 반환 값은 프레임이 정상적으로 읽혔는지 여부
    # 두 번째 반환 값은 읽어온 프레임
    check, frame = cam.read()

    # 읽어온 프레임을 화면에 출력
    cv2.imshow("webcam", frame)

    # 키보드 입력을 확인합니다. 입력이 있으면 루프를 종료
    key = cv2.waitKey(1)
    if key != -1:
        break

# 웹캠 연결을 종료하고, 모든 OpenCV 창을 닫기
cam.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## 웹캠으로 YOLO v5 실행하기(COCO dataset)

- https://github.com/ultralytics/yolov5

### 1. 필요한 라이브러리 불러오기

In [2]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
"""
Run inference on images, videos, directories, streams, etc.

Usage - sources:
    $ python path/to/detect.py --weights yolov5s.pt --source 0              # webcam
                                                             img.jpg        # image
                                                             vid.mp4        # video
                                                             path/          # directory
                                                             path/*.jpg     # glob
                                                             'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                                                             'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

Usage - formats:
    $ python path/to/detect.py --weights yolov5s.pt                 # PyTorch
                                         yolov5s.torchscript        # TorchScript
                                         yolov5s.onnx               # ONNX Runtime or OpenCV DNN with --dnn
                                         yolov5s.xml                # OpenVINO
                                         yolov5s.engine             # TensorRT
                                         yolov5s.mlmodel            # CoreML (macOS-only)
                                         yolov5s_saved_model        # TensorFlow SavedModel
                                         yolov5s.pb                 # TensorFlow GraphDef
                                         yolov5s.tflite             # TensorFlow Lite
                                         yolov5s_edgetpu.tflite     # TensorFlow Edge TPU
"""
import cv2
import argparse
import os
import sys
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn

# FILE = Path(__file__).resolve()
# ROOT = FILE.parents[0]  # YOLOv5 root directory
# if str(ROOT) not in sys.path:
#     sys.path.append(str(ROOT))  # add ROOT to PATH
# ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync
from utils.augmentations import letterbox
import numpy as np

C:\Users\sangwoo\anaconda3\envs\pytorch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 실시간 예측하기(일반카메라)

#### 객체 검출 및 분류를 수행하기 전에 필요한 여러 설정값들을 초기화

In [1]:
imgsz = (640, 640)  # 이미지의 크기 (너비, 높이)
weights = "./yolov5m.pt"  # 학습된 모델의 가중치 파일 경로
device = ''  # 연산에 사용할 장치 (비워 둘 경우, 자동으로 GPU 또는 CPU를 선택)
visualize = False  # 시각화 옵션
augment = False  # 데이터 증강 옵션
conf_thres = 0.25  # 객체 탐지에 사용할 최소 신뢰도 임계값
iou_thres = 0.45  # NMS를 위한 IoU(Intersection over Union) 임계값
classes = [i for i in range(1000)]  # 탐지할 클래스 인덱스 목록
agnostic_nms = False  # 클래스에 관계없이 NMS를 적용할지 여부
max_det = 1000  # 한 이미지에서 탐지할 수 있는 최대 객체 수
webcam = False  # 웹캠을 사용하여 실시간 객체 탐지를 수행할지 여부
view_img = True  # 탐지 결과를 화면에 표시할지 여부
save_crop = False  # 탐지된 객체를 자른 이미지로 저장할지 여부
line_thickness = 5  # 객체 경계 상자의 선 두께
save_txt = False  # 탐지 결과를 텍스트 파일로 저장할지 여부
hide_labels = False  # 탐지된 객체의 레이블을 숨길지 여부
hide_conf = False  # 탐지된 객체의 신뢰도를 숨길지 여부
dnn = False  # DNN 모듈을 사용할지 여부
data = False  # 데이터셋을 사용할지 여부
half = False  # 반정밀도(FP16) 계산을 사용할지 여부
save_img = 'video'  # 이미지 또는 비디오를 저장할지 여부

### 사전 학습된 가중치를 불러와 객체 검출 수행

In [3]:
# 모델 불러오기
device = select_device(device)
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half) # weights에 경로 넣기
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # 이미지 크기 확인 및 업데이트

# 웹캠 초기화
cap = cv2.VideoCapture(0)

while True:
    # 웹캠에서 프레임 캡처
    ret_val, img0 = cap.read()

    bs = 1  # 배치 크기
    vid_path, vid_writer = [None] * bs, [None] * bs

    model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # 워밍업
    dt, seen = [0.0, 0.0, 0.0], 0

    # 패딩된 리사이즈
    img = letterbox(img0, imgsz, stride=stride, auto=pt)[0]

    # 변환
    img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img = np.ascontiguousarray(img)

    im = img.copy()
    im0s = img0.copy()

    t1 = time_sync()
    im = torch.from_numpy(im).to(device)
    im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # 배치 차원 확장
    t2 = time_sync()
    dt[0] += t2 - t1

    # 추론
    visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
    pred = model(im, augment=augment, visualize=visualize)
    t3 = time_sync()
    dt[1] += t3 - t2

    # NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
    dt[2] += time_sync() - t3

    # 예측 결과 처리
    for i, det in enumerate(pred):  # 이미지 당
        seen += 1
        if webcam:  # 배치 크기 >= 1
            p, im0, frame = None, im0s[i].copy(), None
            s += f'{i}: '
        else:
            p, im0, frame = None, im0s.copy(), None
        
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # 정규화 획득 whwh
        imc = im0.copy() if save_crop else im0  # save_crop 용
        annotator = Annotator(im0, line_width=line_thickness, example=str(names))
        if len(det):
            # 박스 좌표를 img_size에서 im0 크기로 변환
            det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

            # 결과 출력
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # 클래스별 탐지 수
                # s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # 문자열 추가

            # 결과 저장
            for *xyxy, conf, cls in reversed(det):
                if save_txt:  # 파일에 저장
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # 정규화된 xywh
                    line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # 라벨 형식
                    with open(f'{txt_path}.txt', 'a') as f:
                        f.write(('%g ' * len(line)).rstrip() % line + '\n')

                if save_img or save_crop or view_img:  # 이미지에 bbox 추가
                    c = int(cls)  # 정수 클래스
                    label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                    annotator.box_label(xyxy, label, color=colors(c, True))
                if save_crop:
                    save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

        # 결과 스트리밍
        im0 = annotator.result()
        if view_img:
            cv2.imshow(str(p), im0)
            key = cv2.waitKey(1)

            if key != -1:
                break

cv2.destroyAllWindows()
cv2.waitKey(1)

YOLOv5  v7.0-153-gff6a9ac Python-3.8.16 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients


KeyboardInterrupt: 